In [1]:
import os
import polars as pl
import altair as alt

In [2]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).collect()

In [4]:
df.filter(pl.col("obdobi") == 2021).select(pl.col("pocet_slov")).sum().item()

15660818

In [5]:
df.filter(pl.col("obdobi") == 2021).select(pl.col("pocet_slov")).sum().item() / (105 * 60 * 8 * 5)

62.14610317460318

## Skokani

In [7]:
import re

In [8]:
def ocisti(frejm):
    return frejm.with_columns(pl.col('text').str.replace_all(r'[^\w\s]','',).str.to_lowercase())

In [9]:
def zgrupni_a_pores_cetnost(frejm):

    frejm_pracovni = frejm.with_columns(pl.col('text').str.split(' ')).explode('text').group_by('text').len().sort(by='len',descending=True)
    celkem = frejm_pracovni.select(pl.col('len')).sum().item()
    
    return frejm_pracovni.with_columns((pl.col('len') / celkem).alias('podil'))

In [10]:
stara_snemovna = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2017)))

In [11]:
stara_snemovna

text,len,podil
str,u32,f64
"""a""",396529,0.03013
"""to""",277500,0.021086
"""se""",253120,0.019233
"""je""",232138,0.017639
"""v""",220998,0.016792
"""že""",209647,0.01593
"""na""",187316,0.014233
"""o""",125001,0.009498
"""tak""",101117,0.007683


In [12]:
nova_snemovna = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2021)))

In [13]:
porovnani = stara_snemovna.rename(
    {'podil':'podil_2017','len':'pocet_2017'}
).join(
    nova_snemovna.rename(
        {'podil':'podil_2021','len':'pocet_2021'}
    ), on='text', how='full'
).with_columns(
    (pl.col('podil_2021') / pl.col('podil_2017')).alias('rozdil')
)

In [14]:
porovnani

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""a""",396529,0.03013,"""a""",486667,0.030911,1.025934
"""to""",277500,0.021086,"""to""",374727,0.023801,1.128793
"""se""",253120,0.019233,"""se""",300428,0.019082,0.992148
"""je""",232138,0.017639,"""je""",272899,0.017334,0.982693
"""v""",220998,0.016792,"""v""",259278,0.016468,0.980708
"""že""",209647,0.01593,"""že""",245624,0.015601,0.979365
"""na""",187316,0.014233,"""na""",233299,0.014818,1.041119
"""o""",125001,0.009498,"""o""",134631,0.008551,0.900314
"""tak""",101117,0.007683,"""tak""",121704,0.00773,1.006104


In [15]:
hranice = 30
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=True)

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""zmáčkne""",36,0.000003,"""zmáčkne""",2124,0.000135,49.319008
"""2026""",35,0.000003,"""2026""",1137,0.000072,27.155309
"""ztišení""",39,0.000003,"""ztišení""",1054,0.000067,22.591149
"""2024""",148,0.000011,"""2024""",3661,0.000233,20.67761
"""ukrajině""",90,0.000007,"""ukrajině""",2220,0.000141,20.619246
"""zdražování""",33,0.000003,"""zdražování""",782,0.00005,19.808662
"""ukrajiny""",79,0.000006,"""ukrajiny""",1752,0.000111,18.538276
"""stiskne""",81,0.000006,"""stiskne""",1531,0.000097,15.799833
"""2025""",146,0.000011,"""2025""",2697,0.000171,15.441533


In [16]:
hranice = 100
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=True)

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""2024""",148,0.000011,"""2024""",3661,0.000233,20.67761
"""2025""",146,0.000011,"""2025""",2697,0.000171,15.441533
"""energií""",173,0.000013,"""energií""",3063,0.000195,14.800051
"""2023""",280,0.000021,"""2023""",4641,0.000295,13.855298
"""zvedne""",202,0.000015,"""zvedne""",2750,0.000175,11.380036
"""teďka""",103,0.000008,"""teďka""",1358,0.000086,11.021098
"""fialy""",253,0.000019,"""fialy""",2729,0.000173,9.016653
"""místopředsedkyně""",474,0.000036,"""místopředsedkyně""",5025,0.000319,8.861761
"""korespondenční""",275,0.000021,"""korespondenční""",2877,0.000183,8.745195


In [17]:
hranice = 100
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=False).with_columns(1 / pl.col("rozdil"))

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil,literal
str,u32,f64,str,u32,f64,f64,f64
"""klaus""",793,0.00006,"""klaus""",103,0.000007,0.108574,9.210297
"""faltýnek""",1047,0.00008,"""faltýnek""",156,0.00001,0.124549,8.028969
"""testy""",678,0.000052,"""testy""",111,0.000007,0.136853,7.307089
"""okd""",684,0.000052,"""okd""",118,0.000007,0.144208,6.934446
"""nesouhlasné""",2064,0.000157,"""nesouhlasné""",358,0.000023,0.144989,6.897065
"""čssd""",1871,0.000142,"""čssd""",345,0.000022,0.154137,6.487724
"""zahájil""",3737,0.000284,"""zahájil""",701,0.000045,0.156804,6.377387
"""bonusu""",688,0.000052,"""bonusu""",134,0.000009,0.162809,6.142163
"""ministryni""",2735,0.000208,"""ministryni""",547,0.000035,0.167183,5.981467


## Skokani od roku 1993

In [19]:
import simplemma

In [20]:
def prvnipad(slovo):
    if len(slovo) > 2:
        return simplemma.lemmatize(slovo, lang='cs')
    else:
        return None

In [ ]:
stare_snemovny = zgrupni_a_pores_cetnost(
    ocisti(df.filter(pl.col("obdobi").is_between(1993,2020)))
).with_columns(
    pl.col("text").map_elements(prvnipad, return_dtype=pl.String).alias("lemma")
).group_by(
    "lemma"
).agg([
    pl.col("len").sum(),
    pl.col("podil").sum()
]).rename(
    {'len':'len_93_21','podil':'podil_93_21'}
)

In [ ]:
snemovna_2021 = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2021))).with_columns(
    pl.col("text").map_elements(prvnipad, return_dtype=pl.String).alias("lemma")
).group_by(
    "lemma"
).agg([
    pl.col("len").sum(),
    pl.col("podil").sum()
]).rename(
    {'len':'len_21_25','podil':'podil_21_25'}
)

In [ ]:
stare_snemovny.sample(10)

In [ ]:
len(stare_snemovny)

In [ ]:
porovnani_93_21 = stare_snemovny.join(
    snemovna_2021, how='full', on='lemma'
).with_columns(
    (pl.col('podil_21_25') / pl.col('podil_93_21')).alias('rozdil')
)

In [ ]:
porovnani_93_21.sort(by='rozdil',descending=True)

In [ ]:
df.filter(pl.col("text").str.contains("[Bb]itcoin")).sort(by='datum').head(10)

In [ ]:
porovnani_93_21.filter(pl.col('lemma_right').str.contains('menstr'))

In [ ]:
porovnani_93_21.filter(pl.col('lemma') == 'bydlení')

In [ ]:
porovnani_93_21.filter((pl.col('len_93_21') >= 5) & (pl.col('len_21_25') >= 10)).sort(by='rozdil',descending=True).head(150)

In [ ]:
narust = {
    "pětikoalice":"(?i)[^\w]pětikoal",
    "energie, elektřina, plyn":"(?i)[^\w](energ[ie]|plyn|elekt[řr])",
    "korespondenční":"(?i)[^\w]korespondenčn",
#    "omikron":"(?i)omikron",
#    "HHC":"(?i)hhc",
#    "kratom":"(?i)kratom",
#    "drahota":"(?i)drahot",
#    "Green Deal":"(?i)green deal",
    "Ukrajina, Ukrajinci, ukrajinský":"(?i)[^\w]ukrajin",
    "Lex Něco/Někdo":"(?i)[^\w]lex \w{0,20}",
#    "Rusko, ruský":"(?i)[^\w]rusk",
#    "Křivoklát":"(?i)křivoklát",
    "menstruace":"(?i)[^\w]menstru[oau]",
#    "bitcoin":"(?i)bitco",
#    "kryptoměny":"(?i)kryptom",
    "bydlení":"(?i)[^\w](bydle[tn])",
#    "dozimetr":"(?i)dozimet",
    "inflace, zdražování":"(?i)[^\w](infla[cč]|zdraž)"
}

In [ ]:
maxima = []
for p, n in narust.items():
    maximum = {
        'slovo':p,
        'max':
        df.filter(
            pl.col('obdobi') == 2021
        ).filter(
            pl.col('text').str.contains(n)
        ).with_columns(
            pl.col('text').str.count_matches(n).alias('pocet_vyskytu')
        ).group_by_dynamic(
            index_column='datum',every='1y'
        ).agg(
            pl.col('pocet_vyskytu').sum()
        ).select(
            pl.col('pocet_vyskytu').max()
        ).item()
    }
    maxima.append(maximum)

maxima = pl.DataFrame(maxima).sort(by='max',descending=True).select(pl.col('slovo')).to_series().to_list()

In [ ]:
slov_po_roce = df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('pocet_slov').sum())
slov_po_ctvrtleti = df.group_by_dynamic(index_column='datum',every='3mo').agg(pl.col('pocet_slov').sum())

In [ ]:
def cetnost_po_roce(popisek, hledani):
    return df.filter(
        pl.col('text').str.contains(hledani)
    ).with_columns(
        pl.col('text').str.count_matches(hledani).alias('pocet_vyskytu')
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col('pocet_vyskytu').sum()
    ).join(
        slov_po_roce, on='datum', how='right'
    ).with_columns(
        (pl.col('pocet_vyskytu') / pl.col('pocet_slov')).alias('podil')
    ).select(
        pl.col(['datum','podil'])
    ).with_columns(
        pl.lit(popisek).alias('co')
    )

In [ ]:
def cetnost_po_cvrtleti(popisek, hledani):
    return df.filter(
        pl.col('text').str.contains(hledani)
    ).with_columns(
        pl.col('text').str.count_matches(hledani).alias('pocet_vyskytu')
    ).group_by_dynamic(
        index_column='datum',every='3mo'
    ).agg(
        pl.col('pocet_vyskytu').sum()
    ).join(
        slov_po_ctvrtleti, on='datum', how='right'
    ).with_columns(
        (pl.col('pocet_vyskytu') / pl.col('pocet_slov')).alias('podil')
    ).select(
        pl.col(['datum','podil'])
    ).with_columns(
        pl.lit(popisek).alias('co')
    ).with_columns(
        pl.when(pl.col("datum") >= pl.date(2021,10,1)).then(pl.lit("současná Sněmovna")).otherwise(pl.lit("předchozí Sněmovny")).alias("ktera_snemovna")
    )

In [ ]:
do_grafu = pl.DataFrame()
for popis, hled in narust.items():
    do_grafu = pl.concat([do_grafu,cetnost_po_cvrtleti(popis, hled)])

In [ ]:
graf = alt.Chart(
    do_grafu.to_pandas(),
    title=alt.Title(
        ["O čem poslanci mluvili častěji než dřív"],
        subtitle=[
            "Kolik slov bylo ve Sněmovně zapotřebí k tomu, aby zazněl konkrétní výraz.",
            "Pozor na rozdílná měřítka: čím níže v grafu, tím vzácnější výskyt."
        ]
    ),
    width=330,
    height=50
).mark_bar(
    width=1
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None, tickCount=33, labelAngle=270)),
    alt.Y('podil:Q', axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', labelExpr="datum.label == 0 ? '' : floor((1 / datum.label) + 0.5) + ' slov'",  tickCount=2, orient='right', title=None)),
    alt.Row('co:N', sort=maxima, title=None, header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='start', labelOrient='top', labelFont='Asap')),
    alt.Color("ktera_snemovna:N", 
              title=None,
              scale=alt.Scale(
                  range=['#6B8EAA','#ba4c43'
                        ]),
              legend=alt.Legend(
                  title=None,
                  direction="horizontal",
                  orient="top"
              )
    )
).configure_axisX(
    grid=False, domain=False
).configure_axisY(
    grid=True, domain=False
).configure_view(
    stroke='transparent'
).resolve_axis(
    x="independent",
    y="independent"
).resolve_scale(
    y="independent"
)

graf

In [ ]:
cetnost_po_roce("pětikoalice","pětikoal")

In [ ]:
porovnani_93_21.filter(pl.col("lemma") == "lex")

In [ ]:
porovnani_93_21.filter(pl.col('podil_21_25').is_null()).sort(by='podil_93_21',descending=True).head(150)

### O čem se nemluví

In [ ]:
porovnani_93_21.filter(~pl.col('podil_93_21').is_null()).filter(pl.col('len_21_25').is_null()).sort(by='len_93_21',descending=True).head(150)

In [ ]:
porovnani_93_21.filter(~pl.col('podil_93_21').is_null()).filter(~pl.col('len_21_25').is_null()).sort(by='rozdil',descending=False).head(150)